### 도쿄 부동산 사건 데이터 수집 
출처 :  <a href = 'http://www.oshimaland.co.jp'> 오시마랜드 </a>

사이트 정보 : 본 웹사이트는 자살사건, 시체유기사건, 살인사건 등 과거에 불미스러운 사건이 있었던 건물을 모아서 보여줍니다.


In [1]:
import pandas as pd
import numpy as np 

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import time

## 에어비앤비 주요 지역 탐색 

### Inside AIrbne 주요 지역 기준 오시마랜드 데이터 수집 

InsideAirbnb의 geojosn 위치정보 파일 사용

In [2]:
import json
import pandas as pd

In [6]:
geo_df = pd.read_json('data/japan.geojson') # pd.read_json 이용
# listing_data = pd.read_csv('../../Tokyo_Airbnb/data/listings.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')


In [33]:
features = geo_df['features'][:15]

geo_dict = {}
for feature in features:
    region = feature['properties']['nam']
    coordinates = feature["geometry"]['coordinates'][0][0]
    for idx, lat_lng in enumerate(coordinates):
        geo_dict[f'{region}_{idx}'] = lat_lng

print(len(geo_dict))

8500


#### Osaka Station

In [61]:
osaka_station = pd.read_csv('data/osaka_station.csv')
osaka_station

,Ｓｔａｔｉｏｎ
0,Higashimikuni
1,Shin-Osaka
2,Nishinakajima-Minamigata
3,Nakatsu
4,Umeda
...,...
119,Ferry Terminal
120,Nanko-higashi
121,Nanko-guchi
122,Hirabayashi


## 크롤링 
- 크롤링 메서드 생성 

####  1. oshimaland 웹사이트 불러오기


In [89]:
def loadWeb(driver, url):
    # header 추가하기 

    
    # 화면 최대 크기 설정 
    driver.get(url)
    driver.maximize_window()
    #모든 창이 로딩 될떄까지 기다려 줌 
    time.sleep(3)
    #팝업창 화면 전환 후 닫아주기
    multi_window = driver.window_handles
    if(len(multi_window)>1):
        driver.switch_to.window(driver.window_handles[-1])
        driver.close()
        time.sleep(3)
        # 접근 페이지 다시 요청
        driver.switch_to.window(driver.window_handles[-1])
        driver.get(url)
    
    # 전체 지도 축소 
    driver.execute_script("document.body.style.zoom='60%'")    
    # time.sleep(2)




#### 2. 부동산 관련 사건 정보 수집

In [90]:
from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException

def get_fire(driver, tokyo_data_dict, district):
        #전체 창 
        
        map =  driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div')
        # fire 객체 수집 
        fire_object_list = driver.find_element(By.CSS_SELECTOR, '#map-canvas > div.mk-map-view.mk-disable-all-gestures > div > div.mk-annotation-container')
        fire_list = []
        fire_list = fire_object_list.find_elements(By.CSS_SELECTOR, '.map-fire')
        original_window_handle = driver.current_window_handle  # 원래 창의 핸들을 저장


        print('fire 개수: ', len(fire_list))
        if len(fire_list) != 0 :
            for fire in fire_list:
                try: 
                    is_enabled = fire.is_enabled()
                    actions = ActionChains(driver)                 
                    if is_enabled:
                        # 클릭이 가능하며 현재 화면에 표시 될 때 클릭 수행
                        # print('click OK')
                        if fire.is_displayed():
                    
                            actions = ActionChains(driver)                 
                            actions.move_to_element(fire).click().perform()

                            # time.sleep(5)
                            WebDriverWait(driver, 7).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.property-info-date > span')))
                            info_date = driver.find_element(By.CSS_SELECTOR, 'li.property-info-date > span').text
                            info_address = driver.find_element(By.CSS_SELECTOR, 'li.property-info-address').text
                            info_content = driver.find_element(By.CSS_SELECTOR, 'div.popup-property-info > section > ul > li:nth-child(3)').text

                            tokyo_data_dict.append({
                                'district': district,
                                'date': info_date,
                                'address': info_address,
                                'content': info_content
                            })

                            time.sleep(3)
                                    # 새로운 탭이 열렸는지 확인하고, 열린 경우 원래 창으로 다시 돌아옵니다.
                            if len(driver.window_handles) > 1:
                                driver.switch_to.window(original_window_handle)
                        else:
                            pass
                            # print('display NOT')
                    else:
                        pass
                        # print('click not')

                except (NoSuchElementException, NoSuchWindowException, MoveTargetOutOfBoundsException, StaleElementReferenceException, WebDriverException) as e:
                    # 에러 처리
                    print(f"Error occurred: {e}")
                    map.send_keys(Keys.ENTER)

                    

                # 필요한 경우 WebDriver 인스턴스 재초기화 등의 작업 수행
                # driver = initialize_driver()

                continue


#### 3. 장소 검색 후 수집

In [91]:
def search_fire_coordinate(driver, geo_dict = None):
    tokyo_data_dict = []
    # 주요 지역 위도 경도 검색 
    # for key, value in tqdm(geo_dict.items()): # geojson
    for value in osaka_station['Ｓｔａｔｉｏｎ'].values: # geojson
        region_search_tab = driver.find_element(By.CSS_SELECTOR, '#geocoder-text')
        region_search_tab.clear()  # 검색어 clear
        # lng = value[1]
        # lat = value[0]
        # #경도 위도 검색
        # search  = f'{lng} {lat}'
        search = value
        region_search_tab.send_keys(search)
        time.sleep(2)
        # 검색 버튼 클릭 
        search_btn = driver.find_element(By.CSS_SELECTOR, "#geocoder-button")
        search_btn.send_keys(Keys.ENTER)
        # ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (751, 3641)
        time.sleep(8)
        # fire 객체 정보 수집 
        get_fire(driver,tokyo_data_dict, search)



### 크롤링 

In [92]:
if __name__ == "__main__":
    # 드라이버 생성 및 웹 로딩
    driver = webdriver.Firefox()
    url = 'https://www.oshimaland.co.jp/'
    loadWeb(driver, url)

    # 지도를 60% 정도로 축소 뒤 수집 
    search_fire_coordinate(driver)
    

# driver.quit()

fire 개수:  41
Error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



ElementNotInteractableException: Message: Element <div class="mk-map-node-element"> is not reachable by keyboard
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:351:5
webdriverSendKeysToElement@chrome://remote/content/marionette/interaction.sys.mjs:668:13
interaction.sendKeysToElement@chrome://remote/content/marionette/interaction.sys.mjs:642:11
sendKeysToElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:535:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:150:31
